In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

import logging
logging.getLogger().setLevel(logging.INFO)

import copy
import multiprocessing as mp

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash

# from config import config
# from pof.loader.asset_model_loader import AssetModelLoader
# from pof import Component, FailureMode, Task
# from pof.interface.dashlogger import DashLogger
# from pof.interface.layouts import *
# from pof.interface.figures import update_condition_fig, update_cost_fig, update_pof_fig, make_inspection_interval_fig

In [2]:
external_stylesheets=[dbc.themes.BOOTSTRAP]

## Working Semaphore example

In [3]:
class Semaphore:
    def __init__(self, filename='semaphore.txt'):
        self.filename = filename
        with open(self.filename, 'w') as f:
            f.write('done')

    def lock(self):
        with open(self.filename, 'w') as f:
            f.write('working')

    def unlock(self):
        with open(self.filename, 'w') as f:
            f.write('done')

    def is_locked(self):
        return open(self.filename, 'r').read() == 'working'

In [4]:
def long_process():
    if semaphore.is_locked():
        raise Exception('Resource is locked')
    semaphore.lock()
    time.sleep(7)
    semaphore.unlock()
    return datetime.datetime.now()

In [9]:
semaphore = Semaphore()

app = JupyterDash(external_stylesheets=external_stylesheets)

def layout():
    return html.Div([
        html.Button('Run Process', id='button'),
        dcc.Interval(id='interval', interval=500),
        dcc.RadioItems(
            id='lock',
            options=[{'label': i, 'value': i} for i in ['Running...', 'Free']]),
        html.Div(id='output')
    ])

app.layout = layout

@app.callback(
    Output('lock', 'value'),
    [Input('interval', 'n_intervals')])
def display_status(*args):
    return 'Running...' if semaphore.is_locked() else 'Free'

@app.callback(
    Output('output', 'children'),
    [Input('button', 'n_clicks')])
def run_process(*args):
    return 'Finished at {}'.format(long_process())

app.scripts.config.serve_locally = True

if __name__ == '__main__':
    app.run_server(mode='inline', port=8000)

In [11]:
class Sem:
    def __init__(self, filename='sem.txt'):
        self.filename = filename
        self.n_iterations = 1
        self.n = None
        self.active = True
        with open(self.filename, 'w') as f:
            f.write(f"{self.n}")

    def do_work(self):

        self.n = self.n_iterations
        self.n_iterations = self.n * 10
        while self.n < self.n_iterations and self.active is True:
            time.sleep(1)
            self.n = self.n + 1
            with open(self.filename, 'w') as f:
                f.write(str(self.n))
        
        return f"complete at {self.n}"

    def check_n(self):
        return open(self.filename, 'r').read()
    

In [12]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

def layout():
    return html.Div([
        html.Button('Run Process', id='run-button'),
        html.Button('Kill Process', id='kill-button'),
        dcc.Interval(id='interval', interval=10),
        html.Div(id='run-output'),
        html.Div(id='kill-output'),
        html.Div(["Input: ",
        dcc.Input(id='my-input', value='initial value', type='text')]),
        html.Div(
        [
            dcc.Interval(id="progress-interval", n_intervals=0, interval=500),
            dbc.Progress(id="progress"),
        ])
    ])

app.layout = layout

sem = Sem()


@app.callback(
    Output('my-input', 'value'),
    [Input('interval', 'n_intervals')])
def display_status(*args):
    return sem.check_n()

@app.callback(
    Output('run-output', 'children'),
    [Input('run-button', 'n_clicks')])
def run_process(*args):
    return 'Finished at {}'.format(sem.do_work())

app.scripts.config.serve_locally = True

if __name__ == '__main__':
    app.run_server(port=8055, mode='inline')

In [ ]:

# Global Comp

# trigger update from changed cell
    # Poison pill
    # Update comp
    # Reset n_iterations
    

# On n_iteration value change
    # comp -> continue simulation
    # sens_vars -> continute simulation

# Interval
    # Check if n_iterations < comp.n_iterations
    # Change n_iterations

# On n_iterations changing
    # Create Graphs...

In [1]:
import time
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
from dash.exceptions import PreventUpdate
from dash_extensions.enrich import Output, Dash, Trigger
from flask_caching.backends import FileSystemCache

In [ ]:
steps, sleep_time = 100, 0.1
# Create example app.
app = Dash(prevent_initial_callbacks=True, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = html.Div([
    html.Button("Click me", id="btn"), html.Div(id="progress"), html.Div(id="result"),
    dcc.Interval(id="interval", interval=500)
])
# Create a server side resource.
fsc = FileSystemCache("cache_dir")
fsc.set("progress", None)


@app.callback(Output("result", "children"),Input("btn", "n_clicks"))
def run_calculation():
    for i in range(steps):
        fsc.set("progress", str((i + 1) / steps))  # update progress
        time.sleep(sleep_time)  # do actual calculation (emulated by sleep operation)
    return "done"


@app.callback(Output("progress", "children"), Input("interval", "n_intervals"))
def update_progress():
    value = fsc.get("progress")  # get progress
    if value is None:
        raise PreventUpdate
    return "Progress is {:.0f}%".format(float(fsc.get("progress")) * 100)


if __name__ == '__main__':
    app.run_server()

In [ ]:
steps, sleep_time = 100, 0.1
# Create example app.
app = Dash(prevent_initial_callbacks=True, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = html.Div([
    html.Button("Click me", id="btn"), html.Div(id="progress"), html.Div(id="result"),
    dcc.Interval(id="interval", interval=500)
])
# Create a server side resource.
fsc = FileSystemCache("cache_dir")
fsc.set("progress", None)


@app.callback(Output("result", "children"), Trigger("btn", "n_clicks"))
def run_calculation():
    for i in range(steps):
        fsc.set("progress", str((i + 1) / steps))  # update progress
        time.sleep(sleep_time)  # do actual calculation (emulated by sleep operation)
    return "done"


@app.callback(Output("progress", "children"), Trigger("interval", "n_intervals"))
def update_progress():
    value = fsc.get("progress")  # get progress
    if value is None:
        raise PreventUpdate
    return "Progress is {:.0f}%".format(float(fsc.get("progress")) * 100)


if __name__ == '__main__':
    app.run_server(port=5000)

In [2]:
import datetime as dt
import os
import time

import dash
import dash_core_components as dcc
import dash_html_components as html
import numpy as np
import pandas as pd
from dash.dependencies import Input, Output
from flask_caching import Cache

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
cache = Cache(app.server, config={
    'CACHE_TYPE': 'filesystem',
    'CACHE_DIR': 'cache-directory'
})

TIMEOUT = 60

@cache.memoize(timeout=TIMEOUT)
def query_data():
    # This could be an expensive data querying step
    df =  pd.DataFrame(
        np.random.randint(0,100,size=(100, 4)),
        columns=list('ABCD')
    )
    now = dt.datetime.now()
    df['time'] = [now - dt.timedelta(seconds=5*i) for i in range(100)]
    return df.to_json(date_format='iso', orient='split')


def dataframe():
    return pd.read_json(query_data(), orient='split')

app.layout = html.Div([
    html.Div('Data was updated within the last {} seconds'.format(TIMEOUT)),
    dcc.Dropdown(
        id='live-dropdown',
        value='A',
        options=[{'label': i, 'value': i} for i in dataframe().columns]
    ),
    dcc.Graph(id='live-graph')
])


@app.callback(Output('live-graph', 'figure'),
              [Input('live-dropdown', 'value')])
def update_live_graph(value):
    df = dataframe()
    now = dt.datetime.now()
    return {
        'data': [{
            'x': df['time'],
            'y': df[value],
            'line': {
                'width': 1,
                'color': '#0074D9',
                'shape': 'spline'
            }
        }],
        'layout': {
            # display the current position of now
            # this line will be between 0 and 60 seconds
            # away from the last datapoint
            'shapes': [{
                'type': 'line',
                'xref': 'x', 'x0': now, 'x1': now,
                'yref': 'paper', 'y0': 0, 'y1': 1,
                'line': {'color': 'darkgrey', 'width': 1}
            }],
            'annotations': [{
                'showarrow': False,
                'xref': 'x', 'x': now, 'xanchor': 'right',
                'yref': 'paper', 'y': 0.95, 'yanchor': 'top',
                'text': 'Current time ({}:{}:{})'.format(
                    now.hour, now.minute, now.second),
                'bgcolor': 'rgba(255, 255, 255, 0.8)'
            }],
            # aesthetic options
            'margin': {'l': 40, 'b': 40, 'r': 20, 't': 10},
            'xaxis': {'showgrid': False, 'zeroline': False},
            'yaxis': {'showgrid': False, 'zeroline': False}
        }
    }


if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
